In [105]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import operator

In [106]:
#Exploring some basic elements spanning all players
basic = pd.read_csv('Basic_Stats.csv', encoding='utf8')
playerdict = {}
qualities = ('Name', 'Experience', 'Height (inches)', 'Weight (lbs)', 'Position')
for i in range(len(basic['Current Status'])):
    if basic['Current Status'][i] == 'Active':
        tor = {}
        for a in qualities:
            tor[a] = basic[a][i]
        playerdict[basic['Player Id'][i]] = tor

## Exploring passing scores
![passing.png](passing.png)


In [141]:
#Exploring passing
'''
· TD Pass = 4pts
· Every 25 passing yards = 1pts
· 2pt Passing Conversion = 2pts
· Interceptions Thrown = -2pts
'''
passes = pd.read_csv('Career_Stats_Passing.csv', encoding='utf8')
passdict = {}
qualities = ('Games Played', 'Passes Attempted', 'Completion Percentage', 'Pass Attempts Per Game', 'TD Passes', 'Percentage of TDs per Attempts', 'Passes Longer than 20 Yards', 'Ints', 'Int Rate', 'Passing Yards Per Attempt')
for i in range(len(passes['Team'])):
    if passes['Player Id'][i] in playerdict:
        tor = {}
        for a in qualities:
            tor[a] = float(passes[a][i]) if passes[a][i] != '--' else 0
        if passes['Player Id'][i] in passdict:
            s = passdict[passes['Player Id'][i]]
            d = {}
            for key, val in s.items():
                if key != 'times':
                    passdict[passes['Player Id'][i]][key] = ((s[key]*s['times'])+tor[key])/(s['times']+1)
            passdict[passes['Player Id'][i]]['times'] += 1
        else:
            tor['times'] = 1
            passdict[passes['Player Id'][i]] = tor
            passdict[passes['Player Id'][i]]['times'] = 1
pass_scores = {}
for key, val in passdict.items():
    g, att, p_c, _, TD, p_TD, p_20, i, p_i, py = qualities
    p = passdict[key]
    pass_scores[key] = ((p[att]*p[py]*p[p_c])/(p[g]*100)) + (p[TD]*p[p_TD]*4/100) + (2*p[p_20]*p[p_c]/100) - (2*p[i]*p[p_i]/100)
top = sorted(pass_scores.items(), key=operator.itemgetter(1))
top.reverse()
positions = {}
for a in top:
    if playerdict[a[0]]['Position'] not in positions:
        positions[playerdict[a[0]]['Position']] = 1
    else:
        positions[playerdict[a[0]]['Position']] += 1
print(positions)

top_by_position = {}

for key in positions.keys():
    top_by_position[key] = []

for a in top:
    pos = playerdict[a[0]]['Position']
    #Limit it to 12 since there's only 12 people in my league
    if len(top_by_position) < 12:
        top_by_position[pos].append(a)

for key, val in top_by_position.items():
    print('Top', key+'s for passing')
    for a in val[:12]:
        print(playerdict[a[0]]['Position'], '-', playerdict[a[0]]['Name'], '\nScore:', a[1])
    print('')

{'QB': 76, 'WR': 35, 'P': 14, 'RB': 11, 'NT': 1, 'SS': 2, 'TE': 1, 'K': 6, 'CB': 1, 'ILB': 1}
Top QBs for passing
QB - Brees, Drew 
Score: 259.2529357430258
QB - Ryan, Matt 
Score: 238.42819022778642
QB - Roethlisberger, Ben 
Score: 228.8986982248521
QB - Luck, Andrew 
Score: 225.7843593142857
QB - Stafford, Matthew 
Score: 224.71211417717893
QB - Wilson, Russell 
Score: 218.2059489
QB - Rivers, Philip 
Score: 215.1952450723208
QB - Winston, Jameis 
Score: 214.81573125
QB - Cousins, Kirk 
Score: 213.3555520000001
QB - Dalton, Andy 
Score: 213.28632078853047
QB - Brady, Tom 
Score: 213.1507624895974
QB - Prescott, Dak 
Score: 211.65699999999998

Top WRs for passing
WR - Pryor, Terrelle 
Score: 12.074031529411762
WR - McEvoy, Tanner 
Score: 5.071428571428571
WR - Sanu, Mohamed 
Score: 2.1823933333333336
WR - Snead, Willie 
Score: 1.9166666666666667
WR - Meredith, Cameron 
Score: 1.02
WR - Lee, Marqise 
Score: 0.7236467236467236
WR - Kerley, Jeremy 
Score: 0.4271604938271605
WR - Tate, Go

In [94]:
#Exploring rushing
'''
· TD Rush = 6pts
· Every 10 rushing yards = 1pt
· 2pt Rushing Conversion = 2pts
'''
rushes = pd.read_csv('Career_Stats_Rushing.csv', encoding='utf8')
passdict = {}
qualities = ('Games Played', 'Passes Attempted', 'Completion Percentage', 'Pass Attempts Per Game', 'TD Passes', 'Percentage of TDs per Attempts', 'Passes Longer than 20 Yards', 'Ints', 'Int Rate')
for i in range(len(passes['Team'])):
    if passes['Player Id'][i] in playerdict:
        tor = {}
        for a in qualities:
            tor[a] = float(passes[a][i]) if passes[a][i] != '--' else 0
        if passes['Player Id'][i] in passdict:
            s = passdict[passes['Player Id'][i]]
            d = {}
            for key, val in s.items():
                if key != 'times':
                    passdict[passes['Player Id'][i]][key] = ((s[key]*s['times'])+tor[key])/(s['times']+1)
            passdict[passes['Player Id'][i]]['times'] += 1
        else:
            tor['times'] = 1
            passdict[passes['Player Id'][i]] = tor
            passdict[passes['Player Id'][i]]['times'] = 1